In [1]:
import sys
sys.executable
import os

In [2]:
os.environ["JAVA_HOME"] = "/Library/Java/JavaVirtualMachines/adoptopenjdk-8.jdk/Contents/Home"

In [3]:
# %pip install spark-nlp==2.5.5
# %pip install spylon-kernel

In [4]:
import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder\
.appName("Spark NLP")\
.master("local[4]")\
.config("spark.driver.memory","16G")\
.config("spark.driver.maxResultSize", "2G")\
.config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.11:2.5.5")\
.config("spark.kryoserializer.buffer.max","1000M")\
.getOrCreate()

import pandas as pd
import numpy as np
import scipy as sc
import sklearn as sk
import re

from pyspark.ml.feature import CountVectorizer, Tokenizer, RegexTokenizer, StopWordsRemover
from pyspark.ml.classification import LogisticRegression
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.ml import Pipeline
from pyspark.ml.stat import Summarizer
from pyspark.sql import SQLContext
from pyspark.sql import functions as F
from pyspark.sql.types import StringType, StructField, StructType, BooleanType, ArrayType, IntegerType, MapType, FloatType

import sparknlp
from sparknlp.pretrained import PretrainedPipeline
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.embeddings import *

In [5]:
sparknlp.version()

'2.5.5'

In [6]:
# create a toy dataframe with text I'd like to tinker with

df = spark.createDataFrame([['1', 'A dish is also best thrown on a bat. No cylinder is made. The first step is to centre a flat disc of large diameter. The fingers are used as before to press down in the middle and are then drawn towards the edge preceded by a ridge of clay which gradually increases the diameter at each trip.'],                            
                            ['2', 'To me it seems absurd to present someone who is only a beginner with anything heavier to throw than one pound of clay. In fact I would tend to commence with even less. This is because the problem of controlling and centering increases rapidly as the size of the lump becomes heavier. Long, tedious attempts to master a big piece only lead to frustration, disappointment, and material which soon becomes too wet to manage anyway. In my opinion, a smaller wheel and less powerful motor is quite sufficient for most students and for many serious workers too.'],
                            ['3', 'Many kilns have elements running along the bottom as well as the sides and sometimes in the door and back. Even so, pots on the top shelf can easily be far cooler than those placed lower down in the kiln, If one superimposes on this additional discrepancy the ones we have just examined, it is easy to visualise that within the same firing chamber quite startling variations in temperature can occur.'],
                            ['4', 'Pieces which have been painted with slip can usually be picked up with reasonable safety, provided the hands are clean and free of dust, but any which have had pottery colours applied to the surface should be held at points away from the pigment or from the inside. If a colour is inadvertantly smudged the damage can aften be repaired; use a razor blade to scratch away the smear and then very carefully fill in again with the paint brush. Where a slipped surface is chipped or otherwise marked the piece at this stage will be too dry to correct with slip, and one can either try and make good straightaway with the nearest available pottery colour or wait until after the biscuit firing and use colour before dipping the piece in glaze.'],
                            ['5', 'When the pieces are being arranged, it is as well to remember that shelves should be as small as possible, consistent with their usefulness in supporting the ware. This is to avoid splitting the chamber into separate compartments. It is far better to allow the pots to overhang the edges a little and so permit the heat to circulate freely. A twelve inch square internal measurement will do best with a shelf no bigger than ten by ten.'],
                            ['6', 'My 1 favorite place is 2 blocks away. I drink 3 coffees and smell 4 flowers. Then I walk 5 dogs to 6 parks and see 7 friends. At 8 pm I head home and watch 9 movies.']],
                           ['rowkey', 'text'])

df.show()

+------+--------------------+
|rowkey|                text|
+------+--------------------+
|     1|A dish is also be...|
|     2|To me it seems ab...|
|     3|Many kilns have e...|
|     4|Pieces which have...|
|     5|When the pieces a...|
|     6|My 1 favorite pla...|
+------+--------------------+



In [36]:
# document assember 
document_assembler = sparknlp.DocumentAssembler()\
.setInputCol("text")\
.setOutputCol("document")\
.setCleanupMode("shrink_full")

# sentence detector
sentence_detector = sparknlp.annotator.SentenceDetector()\
.setInputCols("document")\
.setOutputCol("sentence")

# tokenizer 
tokenizer = sparknlp.annotator.Tokenizer()\
.setInputCols(["sentence"])\
.setOutputCol("token")\
.setTargetPattern("\S+")\
.addInfixPattern("(\\d+)(-)(\\d+)")\
.addInfixPattern("(.+)(\\))(\\.)")\
.addInfixPattern("(\\p{Alpha}+)(\\;)")\
.addInfixPattern("(/)(\\p{Alpha}+)")\
.addInfixPattern("(\\p{Alpha}+)(/)(\\p{Alpha}+)")\
.addInfixPattern("(\\p{Alpha}+)(\\.)(\\p{Upper}\\p{Alpha}+)")\
.addInfixPattern("(.+)(\\.)\\z")\
.addInfixPattern("(.+)([,:/])\\z")\
.addInfixPattern("(\\()(.+)(\\))")\
.addInfixPattern("(\\()(.+)")\
.addInfixPattern("(.+)(\\))")\
.addException("New York")
# .addInfixPattern("(\\d+)(-)(\\d+)")\ # add this to top of infix pattern list for solution

# context dependent spell checker
spell_checker = sparknlp.annotator.ContextSpellCheckerApproach()\
.setInputCols(["token"])\
.setOutputCol("spell")\
.setLanguageModelClasses(1000)\
.setWordMaxDistance(2)\
.setEpochs(5)\
.setMaxCandidates(10)\
.setClassCount(3)\
.setMaxWindowLen(15)\
.setTradeoff(10)

# finisher
finisher = Finisher().setInputCols("spell")

# pipeline
pipeline = Pipeline().setStages([document_assembler,
                                 sentence_detector,
                                 tokenizer, 
                                 spell_checker,
                                 finisher
                                ])

model = pipeline.fit(df)

In [37]:
lp = LightPipeline(model)

In [38]:
# I think sentences need to be pipped into tokenizer; when you pipe in documents then the costs are equal for entire doc.
lp.fullAnnotate("I have a meeting in a few minutes. The weather today is hot. Why are the costs fixed and not varying by token?")

[{'spell': [Annotation(token, 61, 63, The, {'sentence': '2', 'cost': '372.18146042174266'}),
   Annotation(token, 65, 67, are, {'sentence': '2', 'cost': '372.18146042174266'}),
   Annotation(token, 69, 71, the, {'sentence': '2', 'cost': '372.18146042174266'}),
   Annotation(token, 73, 77, costs, {'sentence': '2', 'cost': '372.18146042174266'}),
   Annotation(token, 79, 83, fixed, {'sentence': '2', 'cost': '372.18146042174266'}),
   Annotation(token, 85, 87, and, {'sentence': '2', 'cost': '372.18146042174266'}),
   Annotation(token, 89, 91, not, {'sentence': '2', 'cost': '372.18146042174266'}),
   Annotation(token, 93, 99, varying, {'sentence': '2', 'cost': '372.18146042174266'}),
   Annotation(token, 101, 102, be, {'sentence': '2', 'cost': '372.18146042174266'}),
   Annotation(token, 104, 109, token?, {'sentence': '2', 'cost': '372.18146042174266'}),
   Annotation(token, 35, 37, The, {'sentence': '1', 'cost': '184.0707741911105'}),
   Annotation(token, 39, 45, weather, {'sentence': '1'

In [39]:
lp.annotate("We met on January 1st at a party; there were 5-7 guests at the party.")

{'spell': ['the',
  'were',
  '.',
  '-',
  '.',
  'guests',
  'at',
  'the',
  'party',
  '.',
  'be',
  'met',
  'on',
  'January',
  '1st',
  'at',
  'a',
  'party',
  ';']}

In [12]:
dfNew = spark.createDataFrame([['1', 'We met on January 1st at a party; there were 5-7 guests at the party.']],
                              ['rowkey', 'text'])
dfNew.show()

+------+--------------------+
|rowkey|                text|
+------+--------------------+
|     1|We met on January...|
+------+--------------------+



In [13]:
result = model.transform(dfNew)
result.head()

Row(rowkey='1', text='We met on January 1st at a party; there were 5-7 guests at the party.', finished_spell=['the', 'are', '.', '-', '.', 'guests', 'at', 'the', 'are', '.', 'be', 'be', 'on', 'January', 'is', 'at', 'a', 'party;'])

In [12]:
# I don't like "5" and "7" being replaced with commas by the spell checker...
# spell_checker.addRegexClass('_NUM_', '[0-9]')

In [13]:
# help(sparknlp.annotator.ContextSpellCheckerApproach().addVocabClass)
# dir(sparknlp.annotator.ContextSpellCheckerApproach)
# sparknlp.annotator.ContextSpellCheckerApproach.getParamValue()

In [40]:
# context dependent spell checker
spell_checker1 = sparknlp.annotator.ContextSpellCheckerApproach()\
.setInputCols(["token"])\
.setOutputCol("spell")\
.setLanguageModelClasses(1400)\
.addVocabClass(label = "test", vocab = ['blaa'], userdist = 3)

# pipeline
pipeline1 =  Pipeline().setStages([document_assembler,
                                   sentence_detector,
                                   tokenizer, 
                                   spell_checker1,
                                   finisher
                                  ])
model = pipeline1.fit(df)
# lp = LightPipeline(model)
# lp.annotate("We met on January 1st at a party; there were 5-7 guests at the party. 9 days later we took a trip.")


# help(sparknlp.annotator.ContextSpellCheckerApproach().addRegexClass)

Py4JJavaError: An error occurred while calling o6965.fit.
: java.io.NotSerializableException: com.johnsnowlabs.nlp.annotators.spell.context.ContextSpellCheckerApproach$$anon$1
Serialization stack:
	- object not serializable (class: com.johnsnowlabs.nlp.annotators.spell.context.ContextSpellCheckerApproach$$anon$1, value: com.johnsnowlabs.nlp.annotators.spell.context.ContextSpellCheckerApproach$$anon$1@5bcc965f)
	- element of array (index: 2)
	- array (class [Lcom.johnsnowlabs.nlp.annotators.spell.context.parser.SpecialClassParser;, size 3)
	- field (class: scala.collection.mutable.WrappedArray$ofRef, name: array, type: class [Ljava.lang.Object;)
	- object (class scala.collection.mutable.WrappedArray$ofRef, WrappedArray(com.johnsnowlabs.nlp.annotators.spell.context.parser.DateToken$@581b9187, com.johnsnowlabs.nlp.annotators.spell.context.parser.NumberToken$@7e45150c, com.johnsnowlabs.nlp.annotators.spell.context.ContextSpellCheckerApproach$$anon$1@5bcc965f))
	at org.apache.spark.serializ